In [57]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import sys
from scipy.optimize import brentq, least_squares
from scipy.stats import norm

sys.path.append("..")

# handy it seems
# https://docs.sympy.org/latest/modules/solvers/solvers.html
from sympy.solvers import solve
from sympy import Symbol
from analytical_option_formulae.option_types.vanilla_option import VanillaOption


# read data
swaption_data = pd.read_csv("../data/Swaption_Data.csv")
df_snN_0 = pd.read_csv("../bootstrap_swap_curve/df_1c.csv")

# use dict comprehension?
tenor_mapping = {
    "6m": 0.5,
    "1Y": 1.0,
    "2Y": 2.0,
    "3Y": 3.0,
    "5Y": 5.0,
    "10Y": 10.0,
}

# swaption_data processing
swaption_data["Expiry"] = swaption_data["Expiry"].map(tenor_mapping)
swaption_data["Tenor"] = swaption_data["Tenor"].map(tenor_mapping)
swaption_data[swaption_data.columns[2:-1]] = (
    swaption_data[swaption_data.columns[2:-1]] / 100
)
swaption_data.columns = swaption_data.columns.str.lower()
# df_snN_0 processing
df_snN_0.rename(
    columns={
        "maturity": "expiry",
        "duration": "tenor",
        "pv_fix_nok": "pvbp",
        "k_rate": "snN_0",
    },
    inplace=True,
)

# DAY COUNT CONVENTION IS 30/360
FULL_YEAR = 360

## setup common func

In [58]:
# why b76 for reporting?
class B76Model:
    """
    A base class used to model Black 76 option model
    ...
    Parameters
    ----------
    S : float
        The current swap rate
    K : float
        The strike swap rate
    r : float
        Disc factor, in this specific case it is PVBP
    sigma : float
        Volatility
    T : float
        expiry period (years)
    """

    def __init__(self, S: float, K: float, r: float, sigma: float, T: float):
        self.S = S
        self.K = K
        self.r = r
        self.sigma = sigma
        self.T = T

        self.F = self.S
        self.d1 = self._calculate_d1()
        self.d2 = self._calculate_d2()
        self.discount_factor = r

    def _calculate_d1(self) -> float:
        return (np.log(self.F / self.K) + self.sigma**2 / 2 * self.T) / (
            self.sigma * np.sqrt(self.T)
        )

    def _calculate_d2(self) -> float:
        return self.d1 - self.sigma * np.sqrt(self.T)

    def calculate_call_price(self) -> float:
        return self.discount_factor * (
            self.F * norm.cdf(self.d1) - self.K * norm.cdf(self.d2)
        )

    def calculate_put_price(self) -> float:
        return self.discount_factor * (
            -self.F * norm.cdf(-self.d1) + self.K * norm.cdf(-self.d2)
        )

In [59]:
def implied_volatility(
    S: float, K: float, r: float, price: float, T: float, options_type: str
) -> float:
    try:
        bs_model = lambda x: B76Model(S, K, r, x, T)
        if options_type.lower() == "payer":
            implied_vol = brentq(
                lambda x: price - bs_model(x).calculate_call_price(), 1e-12, 10.0
            )
        elif options_type.lower() == "receiver":
            implied_vol = brentq(
                lambda x: price - bs_model(x).calculate_put_price(), 1e-12, 10.0
            )
        else:
            raise NameError("Payoff type not recognized")
    except Exception:
        implied_vol = np.nan

    return implied_vol

## data description
Lognormal Implied Volatility for IR Swaptions

Strike (Forward + basis point)

In [60]:
swaption_data

,expiry,tenor,-200bps,-150bps,-100bps,-50bps,-25bps,atm,+25bps,+50bps,+100bps,+150bps,+200bps
0,1.0,1.0,0.91570,0.62030,0.44130,0.31224,0.26182,0.2250,0.2096,0.2140,0.2434,0.27488,30.297
1,1.0,2.0,0.83270,0.61240,0.46570,0.35807,0.31712,0.2872,0.2712,0.2684,0.2851,0.31025,33.523
2,1.0,3.0,0.73920,0.56870,0.44770,0.35745,0.32317,0.2978,0.2829,0.2780,0.2877,0.30725,32.833
3,1.0,5.0,0.55190,0.44640,0.36510,0.30242,0.27851,0.2607,0.2498,0.2456,0.2512,0.26536,28.165
4,1.0,10.0,0.41180,0.35040,0.30207,0.26619,0.25351,0.2447,0.2398,0.2382,0.2425,0.25204,26.355
5,5.0,1.0,0.67800,0.49090,0.38400,0.31485,0.29060,0.2726,0.2604,0.2532,0.2494,0.25320,25.980
6,5.0,2.0,0.57880,0.46410,0.39033,0.33653,0.31531,0.2983,0.2856,0.2765,0.2671,0.26540,26.760
7,5.0,3.0,0.53430,0.44440,0.38180,0.33437,0.31536,0.2998,0.2876,0.2782,0.2667,0.26200,26.150
8,5.0,5.0,0.41990,0.36524,0.32326,0.29005,0.27677,0.2660,0.2573,0.2502,0.2406,0.23570,23.400
9,5.0,10.0,0.34417,0.30948,0.28148,0.25954,0.25136,0.2451,0.2399,0.2356,0.2291,0.22490,22.250


In [61]:
strike_deltas = (
    swaption_data.columns[2:-1]
    .str.replace("bps", "")
    .str.replace("atm", "0")
    .str.replace("+", "")
    .to_numpy()
    .astype(float)
)
strike_deltas = strike_deltas / 1e4
strike_deltas

C:\Users\kilo\AppData\Local\Temp\ipykernel_21196\633418603.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace("+", "")


array([-0.02  , -0.015 , -0.01  , -0.005 , -0.0025,  0.    ,  0.0025,
        0.005 ,  0.01  ,  0.015 ])

In [62]:
# note snN_0 is ATM strike
df_snN_0["atm_vol"] = swaption_data["atm"]

In [63]:
df_snN_0

,expiry,tenor,pvbp,pv_float,snN_0,atm_vol
0,1.0,1.0,0.994400,0.031828,0.032007,0.2250
1,1.0,2.0,1.985294,0.066029,0.033259,0.2872
2,1.0,3.0,2.972386,0.101093,0.034011,0.2978
3,1.0,5.0,4.934070,0.173953,0.035255,0.2607
4,1.0,10.0,9.747887,0.374591,0.038428,0.2447
5,5.0,1.0,0.978517,0.038430,0.039274,0.2726
6,5.0,2.0,1.952145,0.078232,0.040075,0.2983
7,5.0,3.0,2.920444,0.117029,0.040072,0.2998
8,5.0,5.0,4.840612,0.198916,0.041093,0.2660
9,5.0,10.0,9.542492,0.416374,0.043634,0.2451


### calibrate dd

In [64]:
### i have to redefine the function gottverdamt
### brain small i cannot into inheritance
class DDModel:
    """
    Displaced diffusion is extension of Black76 with an additional parameter beta
    ...
    Parameters
    ----------
    S : float
        The current swap rate
    K : float
        The strike swap rate
    r : float
        Disc factor, in this specific case it is PVBP
    sigma : float
        Volatility
    T : float
        expiry period (years)
    beta : float
        Displaced diffusion model parameter (0,1], but lecture notes say [0,1]
        https://ink.library.smu.edu.sg/cgi/viewcontent.cgi?article=6976&context=lkcsb_research
    """

    def __init__(
        self, S: float, K: float, r: float, sigma: float, T: float, beta: float
    ):
        self.S = S
        self.K = K
        self.r = r
        self.sigma = sigma
        self.T = T
        self.beta = beta

        self.F = S  # take as is
        self.adjusted_F = self.F / self.beta
        self.adjusted_K = self.K + ((1 - self.beta) / self.beta) * self.F
        self.adjusted_sigma = self.sigma * self.beta
        self.discount_factor = r  # take as is

        self.d1 = self._calculate_d1()
        self.d2 = self._calculate_d2()

    def _calculate_d1(self) -> float:
        return (
            np.log(self.adjusted_F / self.adjusted_K)
            + 0.5 * self.adjusted_sigma**2 * self.T
        ) / (self.adjusted_sigma * np.sqrt(self.T))

    def _calculate_d2(self) -> float:
        return self.d1 - self.adjusted_sigma * np.sqrt(self.T)

    def calculate_call_price(self) -> float:
        return self.discount_factor * (
            self.adjusted_F * norm.cdf(self.d1) - self.adjusted_K * norm.cdf(self.d2)
        )

    def calculate_put_price(self) -> float:
        return self.discount_factor * (
            self.adjusted_K * norm.cdf(-self.d2) - self.adjusted_F * norm.cdf(-self.d1)
        )

In [65]:
def calculate_DD_vol_err(x, strikes, vols, S, r, sigma, T, options_type):
    err = 0.0
    for i, vol in enumerate(vols):
        if options_type[i] == "payer":
            price = DDModel(S, strikes[i], r, sigma, T, x[0]).calculate_call_price()
        else:
            price = DDModel(S, strikes[i], r, sigma, T, x[0]).calculate_put_price()
        implied_vol = implied_volatility(S, strikes[i], r, price, T, options_type[i])
        err += (vol - implied_vol) ** 2
    return err

In [66]:
df_snN_0

,expiry,tenor,pvbp,pv_float,snN_0,atm_vol
0,1.0,1.0,0.994400,0.031828,0.032007,0.2250
1,1.0,2.0,1.985294,0.066029,0.033259,0.2872
2,1.0,3.0,2.972386,0.101093,0.034011,0.2978
3,1.0,5.0,4.934070,0.173953,0.035255,0.2607
4,1.0,10.0,9.747887,0.374591,0.038428,0.2447
5,5.0,1.0,0.978517,0.038430,0.039274,0.2726
6,5.0,2.0,1.952145,0.078232,0.040075,0.2983
7,5.0,3.0,2.920444,0.117029,0.040072,0.2998
8,5.0,5.0,4.840612,0.198916,0.041093,0.2660
9,5.0,10.0,9.542492,0.416374,0.043634,0.2451


In [67]:
ddm_parameters = {}

for _, row in df_snN_0.iterrows():
    pvbp = row["pvbp"]  # pass pvbp for a given A X B
    expiry = row["expiry"]  # expiry
    tenor = row["tenor"]  # pass tenor for a given A X B; i.e B
    S = row["snN_0"]  # pass SnN(0) for a given A X B; i.e B
    sigma = row["atm_vol"]  # pass SnN(0) ATM Sigma

    # dataframe for strikes idk maybe neater
    df_strike_vol = pd.DataFrame()
    strikes = S + strike_deltas
    filter_condition = (swaption_data["expiry"] == row["expiry"]) & (
        swaption_data["tenor"] == row["tenor"]
    )
    vols = swaption_data[filter_condition].T.to_numpy()[2:-1]
    # construct the dataframe
    df_strike_vol["strikes"] = strikes
    df_strike_vol["vols"] = vols
    df_strike_vol["options_type"] = np.where(
        df_strike_vol["strikes"] >= S, "payer", "receiver"
    )
    initial_guess = [0.96]  # beta
    res = least_squares(
        lambda x: calculate_DD_vol_err(
            x,
            df_strike_vol["strikes"],
            df_strike_vol["vols"],
            S,
            pvbp,
            sigma,
            tenor,
            df_strike_vol["options_type"],
        ),
        initial_guess,
        bounds=(0, 1),
    )
    # reuse eko for pivot table
    key = "{}Yx{}Y".format(expiry, tenor)
    ddm_parameters[key] = {
        "expiry": expiry,
        "tenor": tenor,
        "sigma": sigma,
        "beta": res.x[0],
    }

In [69]:
print(ddm_parameters.keys())

dict_keys(['1.0Yx1.0Y', '1.0Yx2.0Y', '1.0Yx3.0Y', '1.0Yx5.0Y', '1.0Yx10.0Y', '5.0Yx1.0Y', '5.0Yx2.0Y', '5.0Yx3.0Y', '5.0Yx5.0Y', '5.0Yx10.0Y', '10.0Yx1.0Y', '10.0Yx2.0Y', '10.0Yx3.0Y', '10.0Yx5.0Y', '10.0Yx10.0Y'])


In [70]:
df_ddm_parameters = pd.DataFrame(ddm_parameters.values(), index=ddm_parameters.keys())
df_ddm_parameters.index.name = "Swaption"
df_ddm_sigma = df_ddm_parameters.pivot(index="expiry", columns="tenor", values="sigma")
df_ddm_beta = df_ddm_parameters.pivot(index="expiry", columns="tenor", values="beta")

In [71]:
print("DDM Sigma Parameters:")
df_ddm_sigma

DDM Sigma Parameters:


tenor,1.0,2.0,3.0,5.0,10.0
expiry,,,,,
1.0,0.2250,0.2872,0.2978,0.2607,0.2447
5.0,0.2726,0.2983,0.2998,0.2660,0.2451
10.0,0.2854,0.2928,0.2940,0.2674,0.2437


In [72]:
print("DDM Beta Parameters:")
df_ddm_beta

DDM Beta Parameters:


tenor,1.0,2.0,3.0,5.0,10.0
expiry,,,,,
1.0,5.601277e-08,8.735887e-11,3.056679e-08,0.000004,0.000011
5.0,9.691115e-07,3.749507e-06,8.210674e-08,0.000001,0.074808
10.0,4.591353e-07,1.057133e-06,5.521688e-07,0.000116,0.000208


### calibrate SABR

### price swaptions